In [1]:
import os

In [2]:
%pwd

'/Users/olasuaifan/Desktop/University/projects/lipedema_stage_type_classifier/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/olasuaifan/Desktop/University/projects/lipedema_stage_type_classifier'

In [5]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [ ]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import read_yaml, create_directories
import tensorflow as tf
tf.config.run_functions_eagerly(True)
 
 
 


2025-08-26 17:07:06.878948: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "images_lipedema")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time

In [9]:


class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        """
            A class that prepares callbacks for TensorFlow/Keras during model training.

            Callbacks are functions that run automatically during training, like saving models, logging, or stopping early.
        """


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

        """
        This creates a TensorBoard callback:

        timestamp → current date and time for unique log folders.

        tb_running_log_dir → where TensorBoard logs will be stored.

        Uses tf.keras.callbacks.TensorBoard to create the callback.

        Purpose → Helps visualize training metrics like accuracy, loss, etc., in TensorBoard.
        """ 

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        """
        This creates a ModelCheckpoint callback:

        Saves the best model during training.

        Uses the path specified in config.checkpoint_model_filepath.

        save_best_only=True → only saves the best model based on validation loss/accuracy.
        """

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

        """
        Returns both callbacks as a list.

          Later, you’ll pass this list into the model training function like:
        model.fit(X_train, y_train, callbacks=prepare_callback.get_tb_ckpt_callbacks())

        This ensures:

        TensorBoard logs get saved.

        Best model checkpoints get saved.
        """

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.optimizers import Adam


In [19]:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    """
        The Training class manages everything related to training your deep learning model.

        It takes a configuration object (TrainingConfig) that contains:

        Image size

        Batch size

        Dataset path

        Number of epochs

        Model paths

        Whether to use data augmentation
    
    """


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        """
        Loads a pretrained or updated model from a saved path.

        Stores it in self.model for later training.
        """
        self.model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
        )   

    

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        """
        rescale=1./255 → Normalizes image pixels from 0–255 → 0–1 (better for training).

        validation_split=0.20 → Uses 20% of data for validation, 80% for training.
        """
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        """
        target_size → Resizes all images to match the model’s input size.

        batch_size → Number of images per batch.

        interpolation="bilinear" → Smooth scaling when resizing images.
            """
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        """
        Uses ImageDataGenerator to load images from folders.

        Creates self.valid_generator for validation images.

        shuffle=False → Keeps validation data in order for accurate evaluation.
        """
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        """
        If augmentation is enabled:

        Randomly rotates, flips, zooms, and shifts images.

        This makes the model more robust and prevents overfitting.

        If disabled → Uses the same generator as validation (no augmentation).
        """

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        """
                Creates self.train_generator for training images.

        shuffle=True → Randomizes training data order for better learning.
        """
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model): #Saves the trained model to the given path.
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        """
                Calculates how many batches to process per epoch for training and validation.
        """
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )
        """

        Starts model training:

        Uses the training and validation generators.

        Trains for the number of epochs specified in config.

        Uses callbacks (e.g., TensorBoard, checkpoints).
        """
        self.save_model(
            path=self.config.trained_model_path.with_suffix(".keras"),
            model=self.model
        ) #Saves the final trained model.

In [20]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-08-26 17:38:24,005: INFO: common: yaml_file: config/config.yaml loaded successfully!]
[2025-08-26 17:38:24,009: INFO: common: yaml_file: params.yaml loaded successfully!]
[2025-08-26 17:38:24,011: INFO: common: created directory at: artifacts]
[2025-08-26 17:38:24,014: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-08-26 17:38:24,015: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2025-08-26 17:38:24,019: INFO: common: created directory at: artifacts/training]
[2025-08-26 17:38:24,261: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 3 images belonging to 20 classes.
Found 51 images belonging to 20 classes.


/Users/olasuaifan/opt/anaconda3/envs/lipedema/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/Users/olasuaifan/opt/anaconda3/envs/lipedema/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.0190 - loss: 5.1717     [2025-08-26 17:38:53,373: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
3/3 ━━━━━━━━━━━━━━━━━━━━ 29s 13s/step - accuracy: 0.0571 - loss: 6.1058 - val_accuracy: 0.3333 - val_loss: 2.2861
